Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32) # a conversão para float32 se refere à precisão no valor
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1), # informação de entrada, informação de saída
                    nn.ReLU(), # Função retificadora - é zero para números negativos e assume valores da reta identidade para números positivos (não é linear)
                    nn.Linear(neuronios_c1, neuronios_c2), # a informação de saída anterior é a entrada nessa camada
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x): # recebe a informação e propaga para todas as camadas
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 500
NEURONIOS_C2 = 100

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[ 0.2469,  0.1542, -0.0126,  0.3106,  0.3220,  0.2939],
        [ 0.1853, -0.3532,  0.2392, -0.1971,  0.2768,  0.3301],
        [-0.2874,  0.0628, -0.3227, -0.2448, -0.2307, -0.2021],
        ...,
        [ 0.1368, -0.2376,  0.2688, -0.0659,  0.4076,  0.0917],
        [ 0.1252, -0.1998,  0.0610, -0.3549, -0.3782, -0.2437],
        [ 0.3985,  0.1235,  0.0579,  0.3869, -0.1146,  0.1523]],
       requires_grad=True)
Parameter containing:
tensor([ 0.1606, -0.0047,  0.0235, -0.2942, -0.0658,  0.1319, -0.2760,  0.2235,
        -0.1087,  0.3915,  0.3783,  0.2334, -0.2059, -0.4034,  0.3992,  0.1267,
        -0.3011, -0.1920,  0.0766,  0.1731,  0.3860,  0.0279,  0.3103, -0.2469,
        -0.3076, -0.3796, -0.3090,  0.0825,  0.2223, -0.2458,  0.1885, -0.0044,
        -0.2206, -0.0842, -0.3855, -0.1997, -0.4043,  0.2139, -0.3666, -0.3303,
         0.0677,  0.1926, -0.2591,  0.1158,  0.0110, -0.3020, -0.3080, -0.1309,
        -0.2859,  0.2346, -0.2456,  0.2508,  0.1741

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.1471],
        [0.1454],
        [0.1433],
        ...,
        [0.1442],
        [0.1453],
        [0.1464]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO) # Adam é um otimizador que varia a taxa de aprendizado

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train() # indica que a rede está no modo de treino

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 250

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0511)
1 tensor(0.0554)
2 tensor(0.0431)
3 tensor(0.0396)
4 tensor(0.0396)
5 tensor(0.0339)
6 tensor(0.0299)
7 tensor(0.0293)
8 tensor(0.0267)
9 tensor(0.0224)
10 tensor(0.0201)
11 tensor(0.0192)
12 tensor(0.0169)
13 tensor(0.0140)
14 tensor(0.0127)
15 tensor(0.0121)
16 tensor(0.0105)
17 tensor(0.0089)
18 tensor(0.0086)
19 tensor(0.0084)
20 tensor(0.0076)
21 tensor(0.0072)
22 tensor(0.0076)
23 tensor(0.0077)
24 tensor(0.0073)
25 tensor(0.0074)
26 tensor(0.0077)
27 tensor(0.0074)
28 tensor(0.0071)
29 tensor(0.0072)
30 tensor(0.0071)
31 tensor(0.0068)
32 tensor(0.0067)
33 tensor(0.0068)
34 tensor(0.0068)
35 tensor(0.0067)
36 tensor(0.0068)
37 tensor(0.0068)
38 tensor(0.0067)
39 tensor(0.0067)
40 tensor(0.0067)
41 tensor(0.0066)
42 tensor(0.0066)
43 tensor(0.0065)
44 tensor(0.0065)
45 tensor(0.0064)
46 tensor(0.0064)
47 tensor(0.0064)
48 tensor(0.0063)
49 tensor(0.0063)
50 tensor(0.0063)
51 tensor(0.0062)
52 tensor(0.0062)
53 tensor(0.0062)
54 tensor(0.0062)
55 tensor(0.0062)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino) # reverte a normalização
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1458.16792107]
[1409.99999118] [1525.13140965]
[1128.99997044] [1172.66653824]
[12991.99967575] [11841.0896759]
[14720.0001564] [14804.26352692]
[2938.99994564] [2717.0813961]
[6635.00014687] [7983.86507034]
[11215.99973297] [9439.14208603]
[1040.99999189] [1443.48042631]
[5040.00003624] [5186.37474823]
[9541.99988747] [8500.89586449]
[3792.99988461] [5410.30333519]
[1002.0000155] [757.31337547]
[2542.00005341] [1549.65122032]
[6788.00001717] [6683.21017647]
[9386.00025749] [8080.97140312]
[1245.99996853] [1733.54594183]
[997.00000262] [913.72890377]
[708.00000262] [700.375579]
[1435.99997544] [1415.50196218]
[2590.00006676] [2549.46569014]
[1392.99997497] [1664.32036877]
[11755.99946976] [9622.32476425]
[2007.99997377] [1371.69869089]
[2707.99998474] [2619.75801897]
[631.00000417] [659.67180443]
[652.99999881] [997.1387043]
[2492.00006247] [2638.06664038]
[594.99999416] [549.26499367]
[760.0000056] [1103.17196798]
[1179.00003028] [1104.81695747]
[1789.9999361] [1761.1

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval() # indica que a rede está no modo de avaliação

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [839.87973475]
[628.00000334] [595.74176908]
[442.00000328] [704.30315161]
[2284.99997044] [2580.07211351]
[1040.99999189] [830.11311781]
[809.99999654] [1225.08088923]
[3049.99987316] [3491.40910721]
[720.00000596] [613.63828182]
[877.99999249] [637.4550066]
[1292.00000429] [1610.83540344]
[7056.00004578] [4909.17755127]
[4176.99999142] [5871.53926086]
[13701.00020599] [14608.53279114]
[2948.0000515] [4067.89324188]
[16732.99985504] [14287.74592209]
[1815.00000048] [1953.61251736]
[16220.00016022] [12089.50086594]
[5967.00019073] [4651.69873047]
[473.00000042] [608.63537502]
[1078.9999795] [1051.78992128]
[452.00000319] [610.28449869]
[801.99998283] [754.62560844]
[2458.99994993] [2530.00638914]
[2492.99998236] [2829.39213371]
[582.00000203] [685.21168911]
[1405.9999671] [1430.42117763]
[8715.99991035] [11538.80628586]
[7634.00004578] [6653.17895508]
[506.99999839] [611.92383814]
[665.99999094] [738.3487978]
[1815.00000048] [1579.45974731]
[707.00001383] [659.01708841]


E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False) # calculo do RMSE que tem a mesma unidade do target utilizado
print(f'Loss do teste: {RMSE}')

Loss do teste: 1376.0903977687265


## Conclusão



Este experimento apresenta o treinamento de uma rede neural pelo pytorch.

O pytorch tem uma rede de treino e avaliação, então é importante indicar que se a rede está no modo de treino ou avaliação porque as ações dentro da rede podem ou não ser alteradas, como o gradiente local.

A rede neural tem preferência quando os dados estão entre 0 e 1 ou -1 e 1, ou seja, dados normalizados.

Classes podem ser projenitores, chamada de classe mãe, e a classe que herda parte da construção da classe mãe é a classe filha.

O pytorch é mais eficiente do que o experimento anterior. Nele há métodos definidos para cada passo do processo e outras funções que podem ser aplicadas como a função retificadora que anula os valores negativos das camadas.

No treinamento de uma rede neural, parte dos dados é reservado para teste, outra parte para validação e um parte para treino. Os dados de validação são utilizados como métrica na rede e o critério de parada é o ponto de mínimo, isso evita que a rede decore os dados, retornando um loss zero que não é o valor mais preciso, mas sim o próprio valor dos dados.

## Playground

